# Which weekday is the best for purchasing VTI (Vanguard Total Stock Market ETF)?

For each weekday defined by weekday 0 = Monday, weekday 1 = Tuesday, weekday 2 = Wednesday, weekday 3 Thursday, weekday 4 = Friday, compare the average price changes of VTI and find out on which day we should purchase index funds if we want to avoid checking its price frequently or potentially for automation.
This topic is worth investigating because according to the Little Book of Common Sense Investing: The Only Way to Guarantee Your Fair Share of Stock Market Returns, the best strategy for personal finance is to invest in index funds, and I wanted to know if I were to automate this process, on average, which weekday I should make my program to purchase VTI. Also, I read a series of books, Sapiens: A Brief History of Humankind, Homo Deus: A Brief History of Tomorrow, and 21 Lessons for the 21st Century written by Yuval Noah Harari and one of the arguments the author makes is that the stock market is the largest and the most complex algorithm humans have invented and I found this idea fascinating and wanted to play with the stock market's data.

## Sampling:
All data is based on the stock markets' data hosted by Yahoo! Finance (https://finance.yahoo.com/quote/VTI/history?period1=1583107200&period2=1583539200&interval=1d&filter=history&frequency=1d).
I extracted data to be used for analysis from the CSV file I downloaded (attached) in the following ways: 

1. Collect data of 70 weeks (70 Mondays, Tuesdays, Wednessday, Thursdays and Fridays) assuming that there is no bias.

2. Based on the assumption that election results affect the stock market a lot, use 70 weeks of data prior to the democrat's primary election (~ Jan 31st, 2020).

3. The news of coronavirus seems to have a huge effect on the stock prices. So in order to mitigate this bias, use 70 weeks of data prior to the coronavirus outbreak (~ Nov 29th, 2019).

4. Ignore the effect of some breaking news based on the assumption that there are always some, take 500 weeks of data.

**To make the sample unbiased, I will remove weeks that include holidays during the week, because, for example, if Monday is a holiday, then Tuesday might kind of function as Monday and this can be considered as a bias in the sample.**

## Code:

In [1]:
import pandas as pd
from scipy.stats import f
from statistics import variance

In [2]:
# utility functions

def days(year: int, month: int):
    days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    if year % 4 == 0 and month == 2:
        return 29
    else:
        return days[month - 1]

def eval_date(date: str):
    return list(map(int, date.split("-")))

def is_sequence(dates):
    n = dates[0][2]
    for i in range(4):
        c_date = dates[i]
        if c_date[2] != n:
            return False
        if c_date[2] == 1:
            month, year = 0, 0
            if c_date[1] != 1:
                month = c_date[1] - 1
                year = c_date[0]
            else:
                month = 12
                year = c_date[0] - 1
            n = days(year, month)
        else:
            n = c_date[2] - 1
            
    return dates[4][2] == n

def collect_data(mem, vals):
    [f, h, w, t, m] = vals
    mem[0].append(m)
    mem[1].append(t)
    mem[2].append(w)
    mem[3].append(h)
    mem[4].append(f)
    return mem

def extract_weeks_without_holidays(n, remove):
    vtis = pd.read_csv('VTI.csv', usecols=['Date','Open','High','Low','Close'])
    vtis['Date'] = vtis['Date'].apply(eval_date)
    vtis = vtis.iloc[::-1]

    if remove:
        while list(vtis.head(1)['Date']) != [remove]:
            vtis = vtis[1:]

    data = [[], [], [], [], []]

    # remove weeks with holidays
    while len(data[0]) < n:
        if is_sequence(list(vtis.head(5)['Date'])):
            data = collect_data(data, vtis.head(5).values)
            vtis = vtis[5:]
        else:
            vtis = vtis[1:]
    
    return data

def avg(lst):
    return list(map(lambda x : sum(x) / len(x), lst))

def get_high_and_low(data):
    return list(map(lambda weekday : list(map(lambda x : (x[2] + x[3]) / 2, weekday)), data))

def get_close(data):
    return list(map(lambda weekday : list(map(lambda x : x[4], weekday)), data))

def get_diff_open_and_close(data):
    return list(map(lambda weekday : list(map(lambda x : x[4] - x[1], weekday)), data))

def get_diff_high_and_low(data):
    return list(map(lambda weekday : list(map(lambda x : x[2] - x[3], weekday)), data))

def sst(avgs, n):
    avg = sum(avgs) / len(avgs)
    res = 0
    for i in range(len(avgs)):
        res += n * ((avgs[i] - avg) ** 2)
    return res

def s_squares(lst):
    res = []
    for i in range(len(lst)):
        s = 0
        avg = sum(lst[i]) / len(lst[i])
        for j in range(len(lst[i])):
            s += (lst[i][j] - avg) ** 2
        res.append(s / (len(lst[i]) - 1))
    return res

def sse(lst):
    ss = s_squares(lst)
    res = 0
    for i in range(len(ss)):
        res += (len(lst[0]) - 1) * ss[i]
    return res

def mst(avgs, n):
    return sst(avgs, n) / (len(avgs) - 1)

def mse(lst):
    return sse(lst) / ((len(lst) * len(lst[0])) - 1)

def test_stat(lst, avgs):
    return mst(avgs, len(lst) * len(lst[0])) / mse(lst)

def test_many_cases(data):
    funcs = [
        # ["(high + low) / 2", get_high_and_low],
        # ["close", get_close],
        ["close - open", get_diff_open_and_close],
        # ["high - low", get_diff_high_and_low]
    ]
    
    for i in range(len(funcs)):
        # print(funcs[i][0])
        print("Average: ", avg(funcs[i][1](data)))
        variances = list(map(variance, funcs[i][1](data)))
        print("Variances: ", variances)
        if max(variances) / min(variances) < 2 and len(data[0]) > 30:
            t = test_stat(funcs[i][1](data), avg(funcs[i][1](data)))
            ndf = len(data) - 1
            ddf = (len(data[0]) - 1) * len(data)
            pvalue = 1 - f.cdf(t, ndf, ddf)
            print("1: all sample sizes > 30 and variances are about the same, so we can apply the test statistics")  
            print("2: Hypothesis\n\tH0: u0 = u1 = u2 = u3 = u4\n\tH1: u's are different")
            print("3: a = 0.05")
            print("4: test-statistics = F distribution with ndf = ", ndf, " ddf = ", ddf, "\n\t", t, " and p-value = ", pvalue)
            print("5: Conclusion")
            if pvalue < 0.05:
                print("\tThe samples provide sufficient evidence to reject H0 and conclude that the average stock price changes differ for different weekdays.")
            else:
                print("\tThe samples provide insufficent evidence to reject H0. So, we conclude that the average stock price changes are the same for all weekdays.")
        else:
              print("test statistics cannot be applied because variances differ: max(v) / min(v) = ", max(variances) / min(variances))
        print("\n")
        
def test(n, remove):
    test_many_cases(extract_weeks_without_holidays(n, remove))

## Hypothesis tests:
Let u0 = Average price changes of VTI on Mondays, u1 = Average price changes of VTI on Tuesdays, …).

### Sample 1:
80 weeks until 3/6/2020

In [3]:
test(80, False)

Average:  [-0.07587359999999847, 0.018250087500000234, -0.09174878749999742, 0.12925057499999984, -0.023375175000001393]
Variances:  [0.6974504482098111, 1.2897918319858226, 1.280393703321207, 0.883689795591841, 1.0972535911117671]
1: all sample sizes > 30 and variances are about the same, so we can apply the test statistics
2: Hypothesis
	H0: u0 = u1 = u2 = u3 = u4
	H1: u's are different
3: a = 0.05
4: test-statistics = F distribution with ndf =  4  ddf =  395 
	 3.0197894596091723  and p-value =  0.017893590381042546
5: Conclusion
	The samples provide sufficient evidence to reject H0 and conclude that the average stock price changes differ for different weekdays.




With this data, since Wednesday has the largest drop,
one could argue that right before closing on Wednesdays and right after opening on Thursdays are the best time to
purchase VTI.

### Sample 2:
80 weeks prior to the democrat's primary (~ Jan 31st, 2020).

In [4]:
test(80, [2020, 1, 31])

Average:  [-0.15662349999999847, 0.10537508749999987, -0.02974907499999766, 0.14262564999999955, -0.1460001500000015]
Variances:  [0.7515647486428079, 0.8673575027480283, 1.4190767345077133, 0.7732300546797766, 1.0465256941546868]
1: all sample sizes > 30 and variances are about the same, so we can apply the test statistics
2: Hypothesis
	H0: u0 = u1 = u2 = u3 = u4
	H1: u's are different
3: a = 0.05
4: test-statistics = F distribution with ndf =  4  ddf =  395 
	 7.980177358681236  and p-value =  3.4048591438518017e-06
5: Conclusion
	The samples provide sufficient evidence to reject H0 and conclude that the average stock price changes differ for different weekdays.




With this data, since Monday has the largest drop,
one could argue that right before closing on Mondays and right after opening on Tuesdays are the best time to
purchase VTI.

### Sample 3:
Before the outbreak of coronavirus (~ Nov 29th, 2019).

In [5]:
test(80, [2019, 11, 29])

Average:  [-0.18562397499999933, 0.12012518750000005, -0.04049907499999748, 0.015625599999999372, -0.08087486250000069]
Variances:  [1.005096611078379, 1.2205628335266359, 1.6038073174336622, 1.1164147996534834, 1.1057329206529316]
1: all sample sizes > 30 and variances are about the same, so we can apply the test statistics
2: Hypothesis
	H0: u0 = u1 = u2 = u3 = u4
	H1: u's are different
3: a = 0.05
4: test-statistics = F distribution with ndf =  4  ddf =  395 
	 4.293668921564952  and p-value =  0.002063229972343983
5: Conclusion
	The samples provide sufficient evidence to reject H0 and conclude that the average stock price changes differ for different weekdays.




With this data, since Monday has the largest drop,
one could argue that right before closing on Mondays and right after opening on Tuesdays are the best time to
purchase VTI.

### Sample 4:
500 weeks of data

In [6]:
test(500, False)

Average:  [-0.056559817999999394, 0.023199975999999466, -0.023119735999999634, 0.0031203099999997904, 0.016420026000000035]
Variances:  [0.5269969660519606, 0.6553447707872658, 0.6092301011043985, 0.6377791121830595, 0.5041108998388957]
1: all sample sizes > 30 and variances are about the same, so we can apply the test statistics
2: Hypothesis
	H0: u0 = u1 = u2 = u3 = u4
	H1: u's are different
3: a = 0.05
4: test-statistics = F distribution with ndf =  4  ddf =  2495 
	 4.564876392757413  and p-value =  0.0011276721807924606
5: Conclusion
	The samples provide sufficient evidence to reject H0 and conclude that the average stock price changes differ for different weekdays.




With this data, since Monday has the largest drop,
one could argue that right before closing on Mondays and right after opening on Tuesdays are the best time to
purchase VTI.

## Discussion:

### Conclusion:
All the tested samples provide sufficient evidence to reject the same hypothesis.
So, I think it is reasonable to conclude that the average stock price changes of VTI differ by weekdays.
In 3 of 4 samples, Monday has the largest drop on average and in 1 of 4 samples, Wednesday has the largest drop.
In all samples, Monday and Wednesday have the largest drops in prices. Even in the sample 2 where Wednesday has the largest drop,
Monday has the second largest drop which is close to Wednesday.
Also, it is important to note that the sample 5 should be weighted higher because it has more data.
So, with these reasons, I conclude that right before the closing on Mondays and right after the market's opening on Tuesdays are the best time to purchase VTI.

### Improvements:
I chose to use 70 weeks for sample 1, 2 and 3 because with less than 70 weeks of data, the estimated variances were not close to each other.
However, this number is arbitrary and could be a source of bias.
Having said that I think the last sample, sample 4, is enough to draw the conlusion because it is a huge sample with variances about the same for all weekdays.

To improve the test, I thought about applying Kruskall-Wallis test for samples with not close enough estimated variances.
However, I decided not to and instead to use larger samples until the variances get close enough, because I do not think stock prices from different weekdays are independent which is a requirement for Kruskall-Wallis test.

Also, it is possible that although I removed weeks with holidays to mitigate biases, but in reality, it could have introduced biases, because weeks with holidays exist in real world.
So, a follow up question would be if a week with holidays has different patterns from a week without holidays.
Also, I do not know if this can be answered using statistical analysis of only stock data, but why there is differences between weekdays in average price changes of VTI.
Given that many trades are made by algorithms nowadays, I wonder if the result will be the same if data from old days before the introduction of algorithms to the stockmarket is the same.